In [32]:
import numpy as np
import pandas as pd

In [107]:
# Load data files

itemDf = pd.read_csv("../Data/item.csv")
promotionDf = pd.read_csv("../Data/promotion.csv")
salesDf = pd.read_csv("../Data/sales.csv")
supermarketsDf = pd.read_csv("../Data/supermarkets.csv")

In [ ]:
# NO nan values in data

print(itemDf.isna().any(axis=1).sum())
print(promotionDf.isna().any(axis=1).sum())
print(salesDf.isna().any(axis=1).sum())
print(supermarketsDf.isna().any(axis=1).sum())

In [ ]:
# No duplicates in data

print(itemDf.duplicated().sum())
print(promotionDf.duplicated().sum())
print(salesDf.duplicated().sum())
print(supermarketsDf.duplicated().sum())

#### Item

In [ ]:
itemDf.head()

In [ ]:
itemDf.shape

In [ ]:
itemDf.nunique()

In [ ]:
itemDf.info()

##### invalid format of size variable

In [221]:
# itemDf['valid_size'] = itemDf['size'].str.strip().str.match(r'^\d+(\.\d+)?\s[A-Z]{2}$')

# Initially identify 138 incorrect format instace for size. Then update manually.

In [ ]:
itemDf.head()

#### Checking brand names

In [ ]:
itemDf['brand'].sort_values().unique()

In [ ]:
itemDf.query("brand == 'Edd Og'")

In [ ]:
itemDf.query("brand == 'Eddie'")

In [227]:
# itemDf['brand'] = itemDf['brand'].replace({'Type 2 Shoppec': 'Shoppec', 'Type 2riso': 'Riso', 'Edd Og': 'Eddie'})

# Edd og has to change Eddie
# Observe 'Type 2 Shoppe', 'Type 2riso' and corrected those.

# itemDf.to_csv('../Data/item.csv', index=False)

#### Description entry errors

In [ ]:
grp = itemDf.groupby('brand')['descrption'].apply(list).reset_index(name='descriptions')

for i in range(0,130):
    print(grp.iloc[i,0],"   ::   ",grp.iloc[i,1])

#### Conver LB to OZ

In [ ]:
def transform_size(size):

    size = size.strip()
    match = pd.Series(size).str.match(r'^\d+\s[A-Z]{2}$')
    
    if match.any():
        num, unit = size.split()
        num = float(num)
        if unit == 'LB':
            return f"{num * 16} OZ"
        else:
            return f"{num} OZ"
    else:
        return size


itemDf['size'] = itemDf['size'].apply(transform_size)

#### Duplicate Description

In [3]:
itemDf[itemDf['descrption'].duplicated(keep=False)].head(10)

,code,descrption,type,brand,size
41,9999985260,PRIVATE LABEL COMPLETE PANCAKE MIX,Type 1,Private Label,32.0 OZ
42,9999985261,PRIVATE LABEL COMPLETE PANCAKE MIX,Type 1,Private Label,32.0 OZ
84,7680851613,BARILLA ELBOW,Type 2,Barilla,16.0 OZ
95,7680851917,BARILLA ELBOW,Type 2,Barilla,16.0 OZ
130,1510000013,CREAMETTE VERMICELLI,Type 2,Creamette,16.0 OZ
134,1510000018,CREAMETTE VERMICELLI,Type 2,Creamette,7.0 OZ
139,1510000042,CREAMETTE SPAGHETTI THIN,Type 2,Creamette,32.0 OZ
140,1510000043,CREAMETTE SPAGHETTI,Type 2,Creamette,16.0 OZ
148,1510000162,CREAMETTE SPAGHETTI,Type 2,Creamette,32.0 OZ
149,1510000163,CREAMETTE SPAGHETTI THIN,Type 2,Creamette,16.0 OZ


In [231]:
# There are duplicate description. Hence we have to merge those. But some are different in size hence we have to consider size also.

In [4]:
itemDf[itemDf.duplicated(subset=['descrption', 'size'], keep=False)]

,code,descrption,type,brand,size
41,9999985260,PRIVATE LABEL COMPLETE PANCAKE MIX,Type 1,Private Label,32.0 OZ
42,9999985261,PRIVATE LABEL COMPLETE PANCAKE MIX,Type 1,Private Label,32.0 OZ
84,7680851613,BARILLA ELBOW,Type 2,Barilla,16.0 OZ
95,7680851917,BARILLA ELBOW,Type 2,Barilla,16.0 OZ
235,8692481596,EDDIE VEG ALPHABETS,Type 2,Eddie,12.0 OZ
237,7518100817,EDDIE VEG ALPHABETS,Type 2,Eddie,12.0 OZ
393,9999908441,PRIVATE LABEL VALUE ELBOW MACARONI,Type 2,Private Label Value,64.0 OZ
394,9999908446,PRIVATE LABEL VALUE SPAGHETTI,Type 2,Private Label Value,64.0 OZ
395,9999908451,PRIVATE LABEL VALUE X WIDE EGG NOODLES,Type 2,Private Label Value,16.0 OZ
396,9999981173,PRIVATE LABEL VALUE ELBOW MACARONI,Type 2,Private Label Value,64.0 OZ


In [28]:
itemDf['mapped_code'] = itemDf.groupby(['descrption', 'size'])['code'].transform('min')

code_mapping = dict(zip(itemDf['code'], itemDf['mapped_code']))

itemDf["code"] = itemDf["mapped_code"]

itemDf.drop("mapped_code", axis=1, inplace = True)

itemDf = itemDf.drop_duplicates()

# Map duplicate description to lower code.
# code_mapping will use to change maped code in other data sets.

# itemDf.to_csv('../Data/item.csv', index=False)

# Update item.csv

In [45]:
itemDf[itemDf.duplicated(keep=False)]

,code,descrption,type,brand,size


In [36]:
# This code mapping is affect for promotion.csv and sales.csv since those have code variable. hence we have to map those code variable according to code_mapping

In [47]:
promotionDf["code"] = promotionDf["code"].replace(code_mapping)

salesDf["code"] = salesDf["code"].replace(code_mapping)

In [57]:
# promotionDf.to_csv('../Data/promotion.csv', index=False)
# salesDf.to_csv('../Data/sales.csv', index=False)

#### Promotion

In [58]:
promotionDf = pd.read_csv("../Data/promotion.csv")

In [60]:
promotionDf

,code,supermarkets,week,feature,display,province
0,2700042240,285,91,Not on Feature,Mid-Aisle End Cap,2
1,2700042292,285,92,Interior Page Feature,Not on Display,2
2,2700042274,285,92,Interior Page Feature,Not on Display,2
3,2700042273,285,92,Interior Page Feature,Not on Display,2
4,2700042254,285,92,Interior Page Feature,Not on Display,2
...,...,...,...,...,...,...
351367,9999966721,387,101,Wrap Front Feature,Not on Display,1
351368,9999966720,387,101,Wrap Front Feature,Not on Display,1
351369,3620000507,387,104,Interior Page Feature,Not on Display,1
351370,3620000493,387,104,Interior Page Feature,Not on Display,1


In [59]:
promotionDf.nunique()

code            474
supermarkets    381
week             62
feature           8
display          11
province          2
dtype: int64

In [61]:
promotionDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351372 entries, 0 to 351371
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   code          351372 non-null  int64 
 1   supermarkets  351372 non-null  int64 
 2   week          351372 non-null  int64 
 3   feature       351372 non-null  object
 4   display       351372 non-null  object
 5   province      351372 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 16.1+ MB


In [62]:
promotionDf.describe()

,code,supermarkets,week,province
count,3.513720e+05,351372.000000,351372.000000,351372.000000
mean,6.095909e+09,191.897815,69.844330,1.410332
std,2.884120e+09,111.168403,17.507184,0.491895
min,6.010113e+08,1.000000,43.000000,1.000000
25%,3.620000e+09,94.000000,55.000000,1.000000
50%,4.800105e+09,191.000000,68.000000,1.000000
75%,9.999968e+09,287.000000,85.000000,2.000000
max,9.999986e+09,387.000000,104.000000,2.000000


In [63]:
promotionDf["feature"].unique()

array(['Not on Feature', 'Interior Page Feature', 'Wrap Interior Feature',
       'Wrap Back Feature', 'Interior Page Line Item',
       'Wrap Front Feature', 'Front Page Feature', 'Back Page Feature'],
      dtype=object)

In [64]:
promotionDf["display"].unique()

array(['Mid-Aisle End Cap', 'Not on Display', 'Rear End Cap',
       'Store Rear', 'Front End Cap', 'In-Shelf', 'Store Front',
       'Secondary Location Display', 'In-Aisle', 'Promo/Seasonal Aisle',
       'Side-Aisle End Cap'], dtype=object)

##### Check whether there exist codes that not in item

In [73]:
codes_not_in_items = promotionDf[~promotionDf['code'].isin(itemDf['code'])]
codes_not_in_items

,code,supermarkets,week,feature,display,province


#### Sales

In [65]:
salesDf = pd.read_csv("../Data/sales.csv")

In [66]:
salesDf

,code,amount,units,time,province,week,customerId,supermarket,basket,day,voucher
0,7680850106,0.80,1,1100,2,1,125434,244,1,1,0
1,3620000470,3.59,1,1100,2,1,125434,244,1,1,0
2,1800028064,2.25,1,1137,2,1,108320,244,2,1,0
3,9999985067,0.85,1,1148,2,1,162016,244,3,1,0
4,9999985131,2.19,1,1323,2,1,89437,244,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1048570,1510000011,2.35,1,1307,2,27,43825,346,665448,189,0
1048571,3620000489,0.99,1,1307,2,27,43825,346,665448,189,0
1048572,9999971309,2.00,2,1410,2,27,42140,346,665449,189,0
1048573,3620000250,1.98,2,1415,2,27,53544,346,665450,189,0


In [67]:
salesDf.nunique()

code              765
amount            971
units              30
time             1440
province            2
week               25
customerId     249610
supermarket       376
basket         665450
day               166
voucher             2
dtype: int64

In [68]:
salesDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   code         1048575 non-null  int64  
 1   amount       1048575 non-null  float64
 2   units        1048575 non-null  int64  
 3   time         1048575 non-null  int64  
 4   province     1048575 non-null  int64  
 5   week         1048575 non-null  int64  
 6   customerId   1048575 non-null  int64  
 7   supermarket  1048575 non-null  int64  
 8   basket       1048575 non-null  int64  
 9   day          1048575 non-null  int64  
 10  voucher      1048575 non-null  int64  
dtypes: float64(1), int64(10)
memory usage: 88.0 MB


In [69]:
salesDf["week"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 22, 23, 24, 25, 26, 27, 28], dtype=int64)

In [70]:
salesDf.describe()

,code,amount,units,time,province,week,customerId,supermarket,basket,day,voucher
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,6.059352e+09,1.780470e+00,1.188219e+00,1.543165e+03,1.441374e+00,1.347071e+01,2.003481e+05,1.980431e+02,3.333491e+05,9.119343e+01,2.356245e-02
std,3.155701e+09,5.966503e+00,5.466197e-01,3.833751e+02,4.965514e-01,8.549682e+00,1.247446e+05,1.122828e+02,1.917741e+05,5.983773e+01,1.516815e-01
min,1.111124e+08,-8.280000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,3.620000e+09,9.900000e-01,1.000000e+00,1.259000e+03,1.000000e+00,6.000000e+00,9.211700e+04,1.010000e+02,1.681505e+05,3.900000e+01,0.000000e+00
50%,5.100003e+09,1.500000e+00,1.000000e+00,1.601000e+03,1.000000e+00,1.200000e+01,1.905030e+05,2.000000e+02,3.318470e+05,8.300000e+01,0.000000e+00
75%,9.999982e+09,2.190000e+00,1.000000e+00,1.824000e+03,2.000000e+00,2.300000e+01,3.039840e+05,2.950000e+02,5.000225e+05,1.570000e+02,0.000000e+00
max,9.999986e+09,5.900000e+03,1.000000e+02,2.359000e+03,2.000000e+00,2.800000e+01,5.100270e+05,3.850000e+02,6.654500e+05,1.950000e+02,1.000000e+00


##### Check whether there exist codes that not in item dataset and supermarket no that not in supermarket dataset

In [74]:
codes_not_in_items = salesDf[~salesDf['code'].isin(itemDf['code'])]
codes_not_in_items

,code,amount,units,time,province,week,customerId,supermarket,basket,day,voucher


In [77]:
supermarket_not_in_supermarkets = salesDf[~salesDf['supermarket'].isin(supermarketsDf['supermarket_No'])]
supermarket_not_in_supermarkets

,code,amount,units,time,province,week,customerId,supermarket,basket,day,voucher


##### Handling negative amount values

In [108]:
salesDf.query("amount <= 0")

,code,amount,units,time,province,week,customerId,supermarket,basket,day,voucher
591,3340060109,-0.01,1,1054,2,1,443392,241,385,1,1
1702,3340060108,-0.01,1,2038,2,1,20649,280,1066,1,1
7555,3340060709,0.00,4,1118,2,1,6710,337,4805,3,1
8550,5100015055,0.00,1,1142,2,1,82269,290,5476,4,1
8840,3340060109,-0.01,1,1831,2,1,410790,257,5659,4,1
...,...,...,...,...,...,...,...,...,...,...,...
1014003,9999971297,-0.78,1,1438,2,26,38120,229,644439,181,0
1034773,3620000445,0.00,1,1236,2,27,132141,347,657045,187,1
1034775,3620000442,0.00,1,1236,2,27,132141,347,657045,187,1
1035843,3620000250,-0.84,4,1205,2,27,121348,367,657690,188,0


In [82]:
salesDf.query("code == 3340060109 and day == 1") #385 bas

,code,amount,units,time,province,week,customerId,supermarket,basket,day,voucher
58,3340060109,0.99,1,2051,2,1,121308,365,30,1,0
77,3340060109,0.98,2,1247,2,1,19666,352,44,1,1
120,3340060109,0.99,1,1853,2,1,155367,311,70,1,0
185,3340060109,0.99,1,948,2,1,148908,273,109,1,0
277,3340060109,0.99,1,1603,2,1,18403,287,177,1,0
322,3340060109,0.99,1,1042,2,1,166483,359,206,1,0
359,3340060109,0.99,1,1651,2,1,81224,359,227,1,0
362,3340060109,0.99,1,1951,2,1,104925,359,229,1,0
447,3340060109,0.98,2,1809,2,1,12210,341,285,1,1
501,3340060109,1.98,2,1558,2,1,133012,307,322,1,0


In [109]:
mapping = salesDf[salesDf['amount'] >= 0].set_index(['code', 'province', 'day', 'voucher'])['amount']/salesDf[salesDf['amount'] >= 0].set_index(['code', 'province', 'day', 'voucher'])['units']

def impute_amount(row):
    if row['amount'] <= 0:
        ss = mapping.get((row['code'], row['province'], row['day'], row['voucher']), np.nan)

        if isinstance(ss, pd.Series):
            return ss.mode().iloc[0]*row["units"]
        else:
            return np.nan
    return row['amount']

salesDf['amount'] = salesDf.apply(impute_amount, axis=1)

In [111]:
salesDf.query("code == 3620000250 and day == 188 and province == 2 and voucher == 0 and units == 4")

,code,amount,units,time,province,week,customerId,supermarket,basket,day,voucher
1035843,3620000250,3.96,4,1205,2,27,121348,367,657690,188,0
1036223,3620000250,3.96,4,1850,2,27,440667,354,657920,188,0
1036232,3620000250,3.96,4,1159,2,27,29393,317,657926,188,0
1036440,3620000250,3.96,4,2046,2,27,140585,265,658054,188,0
1036597,3620000250,3.96,4,1751,2,27,122615,375,658147,188,0
1036655,3620000250,3.96,4,919,2,27,109672,270,658184,188,0
1038207,3620000250,3.96,4,1802,2,27,427068,272,659115,188,0
1043570,3620000250,3.96,4,2003,2,27,146176,346,662397,188,0
1045412,3620000250,3.96,4,1638,2,27,33267,302,663522,188,0


In [112]:
salesDf[salesDf.isna().any(axis=1)]

,code,amount,units,time,province,week,customerId,supermarket,basket,day,voucher
19322,2700042238,NaN,1,2342,2,1,408167,308,12503,7,1
21617,5100015055,NaN,1,1119,2,2,82269,347,13975,9,1
35960,3340060110,NaN,1,1646,2,2,129130,304,23275,14,1
44365,9999985488,NaN,1,2217,2,3,19340,354,28480,16,1
47660,3340060108,NaN,1,1856,2,3,166162,344,30456,17,1
...,...,...,...,...,...,...,...,...,...,...,...
987693,3340060980,NaN,1,1309,2,25,20056,378,627374,175,1
1005285,2700042239,NaN,1,1532,2,26,50786,234,638847,179,1
1006149,1800028064,NaN,1,1949,2,26,862,306,639399,180,1
1007054,2700042232,NaN,1,2235,2,26,386675,273,639991,181,1


In [114]:
salesDf = salesDf.dropna(subset=['amount'])

In [116]:
# salesDf.to_csv('../Data/sales.csv', index=False)

#### Supermarkets

In [78]:
supermarketsDf

,supermarket_No,postal-code
0,199,30319
1,200,30134
2,201,30066
3,202,31093
4,203,30542
...,...,...
382,383,40502
383,384,42367
384,385,37160
385,386,29803


In [79]:
supermarketsDf.nunique()

supermarket_No    387
postal-code       299
dtype: int64

In [80]:
supermarketsDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   supermarket_No  387 non-null    int64
 1   postal-code     387 non-null    int64
dtypes: int64(2)
memory usage: 6.2 KB
